In [83]:
import tensorflow as tf
import numpy as np
import sys
import os
import functools
import time
import config
import shutil
from my_tools import define_scope
import my_tools

#To change
from my_load_data import load_dataset_fn 
print("loaded libs")

loaded libs


In [84]:
print (len(os.listdir(config.PIC_SRC_DIR)))


params = {"result_dir": config.RESULT_DIR,
          "learning_rate": config.LR,
          "img_size": config.CNN_IN_HEIGHT,
          "num_channels":config.CNN_IN_CH,
          "num_classes": config.CNN_OUTPUT_SIZE,
          "batch_size": config.BATCH_SIZE,
          "total_steps": 50,
          "keep_probability": config.KEEP_PROB,
          "print_nth_step": 10,
          "train_dataset": os.path.join(config.DATA_DIR,config.TFRECORD_NAMES[0])}
print("loaded params")


100
loaded params


In [93]:
def parser(serialized_data):
    """
    Decode and map data properly
    """
    
    features = {"image" : tf.FixedLenFeature((),tf.string),
                   "class" : tf.FixedLenFeature((),tf.int64),
                   "x1" : tf.FixedLenFeature((),tf.int64),
                   "y1" : tf.FixedLenFeature((),tf.int64),
                   "x2" : tf.FixedLenFeature((),tf.int64),
                   "y2" : tf.FixedLenFeature((),tf.int64),
                   "index" : tf.FixedLenFeature((),tf.int64)}
    parsed_features = tf.parse_single_example(serialized_data, features)
    
    image_string = parsed_features["image"]
    image = tf.decode_raw(image_string, tf.uint8)
    image = tf.cast(image,dtype=tf.float32)
    
    
    label = tf.cast((parsed_features["class"], 
            parsed_features["x1"],
            parsed_features["y1"],
            parsed_features["x2"],
            parsed_features["y2"]), tf.int32)
    index = tf.cast(parsed_features["index"], tf.int32)
    
    return image, label, index

In [94]:
#WRAPPERS FOR MODEL

class Model(object):
    """
    Model of NN
    """
    #Variable of model
    #accuracy = tf.reduce
    
    
### INITIALIZATION ####    
    def __init__(self, params):
        #self.permutation = np.random.permutation(range(len(os.listdir(config.PIC_SRC_DIR))))
        
        self.img_size = params["img_size"]
        self.num_channels = params["num_channels"]
        self.num_classes = params["num_classes"]
        self.lr = params["learning_rate"]
        self.keep_prob = params["keep_probability"]
        self.write_step = params["print_nth_step"]
        self.is_training = False
        self.global_step = tf.train.get_or_create_global_step()
        self.check_mode = 1 # 1 for test ,2 for validation
        #remove bellow
        self.data = tf.placeholder(dtype= tf.float32, shape=[None,256,256,3])
        self.target = tf.placeholder(dtype = tf.int32, shape=[None,5])
        self.index = tf.placeholder(dtype= tf.int32,shape=[None,1])
        self.permutation = np.random.permutation(range(1,6))
        self.data_iter = 1 
        self.total_epoch = 0
        
        #FUNCTIONS, don not remove
        self.data_pipeline
        self.prediction
        self.loss_op
        self.optimize
        self.summary

    

        
#FUNCTION DEFINITIONS
    @define_scope
    def data_pipeline(self):
        """
        loading of TFRecords
        """
        train_dataset = tf.data.TFRecordDataset(params["train_dataset"])
        #test_dataset = tf.data.TFRecordDataset(filenames=config.TFRECORD_NAMES[1],num_parallel_reads= 2)
        train_dataset =train_dataset.map(parser)
        iterator = tf.data.Iterator.from_structure( train_dataset.output_types,
                                                    train_dataset.output_shapes)
        self.data, self.target, self.index = iterator.get_next()
        
        
        self.data = tf.reshape(self.data,[-1, self.img_size, self.img_size, self.num_channels]) 
        
        self.train_init = iterator.make_initializer(train_dataset)
        #test_init = iterator.make_initializer(test_dataset)

        
   
 ##### NEURAL NETWORK #####   
    @define_scope
    def prediction(self, if_training=True):
        """
        Main body of neural network, takes data and labels as input,
        returns feature map of photo
        """
    
        #1 conv layer
        conv1 = tf.layers.conv2d(inputs = self.data, 
                             filters = 32,
                             kernel_size = [5,5],
                             strides = 1,
                             padding = "same",
                             activation = tf.nn.relu)
        
        #1 pool layer, img size reduced by 1/4
        pool1 = tf.layers.max_pooling2d(inputs=conv1,
                                        pool_size = 2, 
                                        strides = 2,
                                        padding = "same")

        #2 conv layer
        conv2 = tf.layers.conv2d(inputs = pool1, 
                             filters = 64,
                             kernel_size = 5,
                             strides = 1,
                             padding = "same",
                             activation = tf.nn.relu)

        #2 pool overal image size reduced totaly by factor of 1/16
        pool2 = tf.layers.max_pooling2d(inputs = conv2,
                                        pool_size = 2, 
                                        strides = 2,
                                        padding = "same")
        
        pool2_flat = tf.reshape(pool2,[-1, 64*64*64])

        dense = tf.layers.dense(inputs = pool2_flat,
                            units = 128,
                            activation = tf.nn.relu)
        dropout = tf.layers.dropout(dense,rate = self.keep_prob,training = self.is_training)
        
        dense2 = tf.layers.dense(inputs = dropout,
                             units = self.num_classes,
                             activation = tf.nn.relu)
       
        return tf.nn.softmax(dense2)
    
    
    
#### LOSS ####    
    @define_scope
    def loss_op(self):
        """
        loss
        """
        self.loss = tf.reduce_mean(
                    tf.nn.softmax_cross_entropy_with_logits_v2(logits = self.prediction,
                                                                labels = self.target))
        
        return self.loss
    
    
    
##### OPTIMIZER #####    
    @define_scope    
    def optimize(self):
        """
        Optimizer of model
        """
        optimizer = tf.train.AdamOptimizer(learning_rate=self.lr)
        
        return optimizer.minimize(self.loss, global_step=self.global_step)
    
    
    
###summary####    
    @define_scope
    def summary(self):
        '''
        Create summaries to write on TensorBoard
        '''
        with tf.name_scope('summaries'):
            tf.summary.scalar('loss', self.loss)
            tf.summary.histogram('histogram_loss', self.loss)
            self.summary_op = tf.summary.merge_all()
            
            
            
#### TRAIN ###        
    def train(self, sess, ckpt_dir, writer, saver, step, epoch):
        """
        Training op for model
        """
        start_time = time.time()
        init=self.train_init
        sess.run(init)
        self.is_training = True
        total_loss = 0
        num_batches = 0
        self.training = True
        try :
            while True:
                #train
                loss, _,summary = sess.run([self.loss_op, self.optimize, self.summary_op])
                writer.add_summary(summary, global_step=step)
                if (step + 1) % self.write_step == 0:
                    print('Loss at step {0}: {1}'.format(step, loss))
                    saver.save(sess, ckpt_dir, global_step = step)
                step += 1
                total_loss += loss
                num_batches += 1
        except tf.errors.OutOfRangeError:
            pass
        print('Average loss at epoch {0}: {1}'.format(epoch, total_loss/num_batches))
        print('Took: {0} seconds'.format(time.time() - start_time))
        
        return step
    
    
    
 #### EVALUATE ####    
    def evaluate(self, init, step, epoch):
        """
        Evaluate once from test
        """
        start_time = time.time()
        self.is_training = False
        try:
            loss, summary = sess.run([self.loss, self.summary_op])
            writer.add_summary(summary, global_step=step)
        except tf.errors.OutOfRangeError:
            pass
        print('Loss at epoch {0}: {1}'.format(epoch, loss))
        print('Took: {0} seconds'.format(time.time() - start_time))
        
        
        
 ### TO DO ####   
    def train_n_times(self, n_times):
        pass
        
    

In [95]:
total_epoch = 7
def main(remove_results = False):
    result_dir = config.RESULT_DIR
    
    if remove_results == True:
        shutil.rmtree(result_dir, ignore_errors=True)
    #Check for dirs, if not present make them
    if not os.path.exists(result_dir):
        os.makedirs(result_dir)
    ckpt_dir=os.path.join(result_dir,"ckpt")
    if not os.path.exists(ckpt_dir):
        os.makedirs(ckpt_dir)
    
    graph = tf.Graph()
    #Get name as default graph
    with graph.as_default():

        print("Starting Session")
        #Assign name to session, assign it's default graph as graph
        with tf.Session(graph=graph) as sess:
            
            #Creating summary writer 
            writer = tf.summary.FileWriter(ckpt_dir, graph=graph)
                
            #Initialization of Model, load all Model functions returning variables
            model = Model(params)
            
            #Assign Initializer
            init = tf.global_variables_initializer()
            
            #Creating save for model session for future saving and restoring model
            saver = tf.train.Saver()
            
            sess.run(init)
            #Loading last checkpoint
            ckpt = tf.train.get_checkpoint_state(result_dir)
            if ckpt and ckpt.model_checkpoint_path:
                #if ckpt found load it and load global step
                saver.restore(sess, ckpt.model_checkpoint_path)
                print("Found checkpoint")
            step = model.global_step.eval()    
                
            print("we are at step {0}".format(step))        
            #Training
            print("Starting Training")

            for epoch in range(1,total_epoch):
                try: 
                    step = model.train(sess, ckpt_dir, writer, saver, step, epoch)
                except KeyboardInterrupt:
                    print("keyboard interrupt")
                    step = model.global_step.eval()
                    break
            #model.global_step = tf.convert_to_tensor(step, dtype=tf.int32)
            saver.save(sess, ckpt_dir, global_step = step)
            print(model.global_step.eval(), step)
        print("Finnished session")
        #Merge all summaries
        #writer.flush()
        writer.add_graph(graph)
        writer.close()
        print("Closed summary, work finnished")

In [96]:
### tf.reset_default_graph()
if __name__ == '__main__':
    check = True
    main(remove_results=check)
    check = False
    main(remove_results=check)

Starting Session
Initialized Model.data_pipeline
Initialized Model.prediction
Initialized Model.loss_op
Initialized Model.optimize
Initialized Model.summary
we are at step 0
Starting Training
Loss at step 9: 687.5054321289062
Loss at step 19: 1041.80126953125
Loss at step 29: 1248.00341796875
Loss at step 39: 877.8466796875
Loss at step 49: 848.272705078125
Loss at step 59: 931.70166015625
Average loss at epoch 1: 890.8946685791016
Took: 86.60795378684998 seconds
Loss at step 69: 687.5054321289062
Loss at step 79: 1041.80126953125
Loss at step 89: 1248.00341796875
Loss at step 99: 877.8466796875
Loss at step 109: 848.272705078125
Loss at step 119: 931.70166015625
Average loss at epoch 2: 890.8947011311849
Took: 77.6024386882782 seconds
Loss at step 129: 687.5054321289062
Loss at step 139: 1041.80126953125
Loss at step 149: 1248.00341796875
Loss at step 159: 877.8466796875
Loss at step 169: 848.272705078125
Loss at step 179: 931.70166015625
Average loss at epoch 3: 890.8947011311849
Too

In [ ]:
"""
tensorboard --logdir="results/"
"""